# U.S. Medical Insurance Costs

**Project Aims**

This project will aim to analyze the U.S. Medical Insurance Costs dataset.

First, I will use python classes and methods to sort and organise the data. I will use containers to store the data and methods to allow analsing various aspects of the data and perform various functions. I will also create methods to analyse the data and group the data.

Secondly, I will use libraries to then query tables and visualise the data in the dataset and create function which will perform analysis on the data.

Lastly, I will create a dashboard to visualise the data.

**1) Imports**

In [32]:
import csv

**2) Creating a Class for handling the data**

In [33]:

class Data:
    def __init__(self, csv_file, name):
        self.name = name
        self.csv_file = csv_file
        self.data = {}
    
    ## Method to read and display the data in the csv class object
    
    def readcsv(self):
        with open(self.csv_file, "r") as file:
            reader = csv.DictReader(file)
            
            person_number = 1
            
            for row in reader:
                age = row["age"]
                sex = row["sex"]
                bmi = row["bmi"]
                children = row["children"]
                smoker = row["smoker"]
                region = row["region"]
                charges = row["charges"]
                
                key = f"Person {person_number}"
                
                self.data[key] = {"Age" : age, "Sex" : sex, "BMI" : bmi, "Children" : children, "Smoker" : smoker, "Region" : region, "Charges" : charges}
                
                person_number += 1
                
                
                


In [34]:
file_name = "insurance.csv"        
medical_data = Data(file_name, "Medical Data")
medical_data.readcsv()
print(medical_data.data.get("Person 1"))

{'Age': '19', 'Sex': 'female', 'BMI': '27.9', 'Children': '0', 'Smoker': 'yes', 'Region': 'southwest', 'Charges': '16884.924'}
